In [1]:
#importing Libraries
import nltk
import re
import csv
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.naive_bayes import MultinomialNB

In [2]:
#Loading Datasets
reviews= pd.read_csv('MLimdbtrain.csv')
test = pd.read_csv('MLimdbtest.csv')
reviews = reviews[['0', '1']]
test = test[['0', '1']]
reviews

,0,1
0,zentropa much common the third man another noi...,1
1,zentropa original movie i've seen years like u...,1
2,lars von trier never backward trying new techn...,1
3,contains spoilers due describe film techniques...,1
4,that first thing sprang mind watched closing c...,1
...,...,...
24995,there isn't enough there funny spots really en...,0
24996,tainted look kibbutz life this film less cultu...,0
24997,saw movie released hailed best picture year is...,0
24998,any film begins cowhand shagging female calf c...,0


In [3]:
reviews['0'].str.lower() #coverting into lowercase
test['0'].str.lower() #coverting into lowercase
#removing special characters
#reviews['0']=reviews['0'].replace(r'[^0-9a-zA-Z]', '', regex=True).replace("?.,!:;$%^&*()#@+/<>=\\[]_~{}|`,')(\"","") 
#test['0']=test['0'].replace(r'[^0-9a-zA-Z ]', '', regex=True).replace("?.,!:;$%^&*()#@+/<>=\\[]_~{}|`,')(\"","")

0        previous reviewer claudio carvalho gave much b...
1        contains spoiler information watch director fi...
2        this first deepa mehta film saw film hindi ver...
3        this great film every sense word tackles subje...
4        stunningly well-made film exceptional acting d...
                               ...                        
24995    this basically run mill violent biker flick co...
24996    caught wild rebels one mystery science theatre...
24997    decide watch wild rebels don't expect anything...
24998    wild rebels probably fun second film drive mov...
24999    probably biggest thing wild rebels hurts hero ...
Name: 0, Length: 25000, dtype: object

In [36]:
#Naive Bayes Model
vectorizer = CountVectorizer()
train_features = vectorizer.fit_transform(reviews['0'])
test_features = vectorizer.transform(test['0'])

nb = MultinomialNB()
nb.fit(train_features, reviews['1'])

# Now we can use the model to predict classifications for our test features.
predictions = nb.predict(test_features)

# Compute the error.  
print(metrics.classification_report(test['1'], predictions))
print("accuracy: {0}".format(metrics.accuracy_score(test['1'], predictions)))


              precision    recall  f1-score   support

           0       0.79      0.88      0.83     12500
           1       0.87      0.77      0.81     12500

    accuracy                           0.82     25000
   macro avg       0.83      0.82      0.82     25000
weighted avg       0.83      0.82      0.82     25000

accuracy: 0.825


In [37]:
#testing the Naive Bayes model
review = """I've recently moved into a new apartment and needed a router to attach to the internet. 
Frankly, I was intimidated by the sheer amount (and price) of routers on the market so it felt like a gamble getting this, 
but I'm glad I did! It was easy to set up, the only trouble I ever had was plugging the ethernet into the wrong outlet (which was completely my bad). 
The signal works amazing, we have it on the main floor and get signal on both the lower and upper levels. 
We've had no connection issues, and everything has run perfectly so far. If you're looking for something inexpensive but functional, 
I'd definitely recommend this!"""

review_vector = vectorizer.transform([review]) # vectorizing

result = int(nb.predict(review_vector))
if result != 1:
  print("Negative")
else:
  print("Positive")

Negative


In [38]:
#testing the Naive Bayes model
review = """I received defective piece display is not working properly"""
review_vector = vectorizer.transform([review]) # vectorizing

result = int(nb.predict(review_vector))
if result != 1:
  print("Negative")
else:
  print("Positive")

Negative


In [39]:
#SVM Model Without transformation and tested using best C obtained below
svmvectorizer = CountVectorizer()
svmtrain_features = svmvectorizer.fit_transform(reviews['0'])
svmtest_features = svmvectorizer.transform(test['0'])

# Perform classification with SVM, kernel=linear
linearsvm = svm.LinearSVC(C = 0.125,max_iter=100000)
linearsvm.fit(svmtrain_features, reviews['1'])
predict_linearsvm = linearsvm.predict(svmtest_features)
# Now we can use the model to predict classifications for our test features.
print(classification_report(test['1'], predict_linearsvm))
print("accuracy: {0}".format(metrics.accuracy_score(test['1'], predict_linearsvm)))


              precision    recall  f1-score   support

           0       0.85      0.86      0.85     12500
           1       0.86      0.84      0.85     12500

    accuracy                           0.85     25000
   macro avg       0.85      0.85      0.85     25000
weighted avg       0.85      0.85      0.85     25000

accuracy: 0.85296


In [16]:
# Transforming the Data
from sklearn.preprocessing import Normalizer
transformer = Normalizer().fit(svmtrain_features)

svmtrain_trans = transformer.transform(svmtrain_features)
transformer = Normalizer().fit(svmtest_features)

svmtest_trans = transformer.transform(svmtest_features)

In [17]:
#Best C 
best_C = None
best_score = 0
accuracy = []
for C in (0.125, 0.25, 0.5, 1, 2, 4, 8, 16, 32, 64, 128, 256, 512):
  linearsvm = svm.LinearSVC(C=C,max_iter=100000)
  linearsvm.fit(svmtrain_trans, reviews['1'])
  score = linearsvm.score(svmtest_trans, test['1'])
  accuracy = accuracy + [score]
  if score > best_score:
    best_score = score
    best_C = C
print(best_C,best_score,accuracy)

0.25 0.88244 [0.87912, 0.88244, 0.88112, 0.87752, 0.87152, 0.86408, 0.85528, 0.84928, 0.8456, 0.84324, 0.8416, 0.84092, 0.84056]


In [23]:
#SVM Performance after using Best C and Transformed data; Improved Accuracy
# Perform classification with SVM, kernel=linear
linearsvm = svm.LinearSVC(C = 0.25,max_iter=100000)
linearsvm.fit(svmtrain_trans, reviews['1'])
predict_linearsvm = linearsvm.predict(svmtest_trans)
# Now we can use the model to predict classifications for our test features.
print(classification_report(test['1'], predict_linearsvm))
print("accuracy: {0}".format(metrics.accuracy_score(test['1'], predict_linearsvm)))

              precision    recall  f1-score   support

           0       0.88      0.88      0.88     12500
           1       0.88      0.88      0.88     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000

accuracy: 0.88244


In [18]:
#testing the SVM model
review = """I've recently moved into a new apartment and needed a router to attach to the internet. 
Frankly, I was intimidated by the sheer amount (and price) of routers on the market so it felt like a gamble getting this, 
but I'm glad I did! It was easy to set up, the only trouble I ever had was plugging the ethernet into the wrong outlet (which was completely my bad). 
The signal works amazing, we have it on the main floor and get signal on both the lower and upper levels. 
We've had no connection issues, and everything has run perfectly so far. If you're looking for something inexpensive but functional, 
I'd definitely recommend this!"""

review_vector = svmvectorizer.transform([review]) # vectorizing

result = int(linearsvm.predict(review_vector))
if result != 1:
  print("Negative")
else:
  print("Positive")

Positive


In [19]:
#testing the SVM model
review = """I received defective piece display is not working properly"""
review_vector = svmvectorizer.transform([review]) # vectorizing

result = int(linearsvm.predict(review_vector))
if result != 1:
  print("Negative")
else:
  print("Positive")

Negative


In [41]:
#Third Model
from sklearn import linear_model
#Logistic Regression
lrvectorizer = CountVectorizer()
lrtrain_features = lrvectorizer.fit_transform(reviews['0'])
lrtest_features = lrvectorizer.transform(test['0'])

#model
lrmodel = linear_model.LogisticRegression(max_iter=10000)

lrmodel.fit(lrtrain_features, reviews['1'])
lrpredict = lrmodel.predict(lrtest_features)

# results
print(classification_report(test['1'], lrpredict))
print("accuracy: {0}".format(metrics.accuracy_score(test['1'], lrpredict)))

              precision    recall  f1-score   support

           0       0.86      0.87      0.86     12500
           1       0.87      0.86      0.86     12500

    accuracy                           0.86     25000
   macro avg       0.86      0.86      0.86     25000
weighted avg       0.86      0.86      0.86     25000

accuracy: 0.86376


In [42]:
#Testing the Logistic Regression model on a Positive Review
review = """I've recently moved into a new apartment and needed a router to attach to the internet. Frankly, I was intimidated by the sheer amount (and price) of routers on the market so it felt like a gamble getting this, but I'm glad I did! It was easy to set up, the only trouble I ever had was plugging the ethernet into the wrong outlet (which was completely my bad). The signal works amazing, we have it on the main floor and get signal on both the lower and upper levels. We've had no connection issues, and everything has run perfectly so far. If you're looking for something inexpensive but functional, I'd definitely recommend this!"""
review_vector = lrvectorizer.transform([review]) # vectorizing

result = int(lrmodel.predict(review_vector))
if result != 1:
  print("Negative")
else:
  print("Positive")

Positive


In [43]:
#Testing the Logistic Regression model on a Negative Review
review = """I received defective piece display is not working properly"""
review_vector = lrvectorizer.transform([review]) # vectorizing

result = int(lrmodel.predict(review_vector))
if result != 1:
  print("Negative")
else:
  print("Positive")

Negative


In [8]:
#vectorize and apply K-Means
kmvectorizer = CountVectorizer()
kmtrain_features = kmvectorizer.fit_transform(reviews['0'])
kmtest_features = kmvectorizer.transform(test['0'])
#print(kmtrain_features)
#print(kmtest_features)

In [13]:
from sklearn.cluster import KMeans 

kmmodel = KMeans(n_clusters=2,n_init=10 ,random_state=5)

kmmodel.fit_predict(kmtrain_features)
#print(kmmodel.labels_)

array([1, 1, 1, ..., 0, 1, 1], dtype=int32)

In [14]:
#Predit on test data
kmmpredict = kmmodel.predict(kmtest_features)

print(classification_report(test['1'], kmmpredict))
print("accuracy: {0}".format(metrics.accuracy_score(test['1'], kmmpredict)))

              precision    recall  f1-score   support

           0       0.51      0.17      0.25     12500
           1       0.50      0.84      0.63     12500

    accuracy                           0.50     25000
   macro avg       0.51      0.50      0.44     25000
weighted avg       0.51      0.50      0.44     25000

accuracy: 0.50336


In [20]:
#k-means on transformed data
from sklearn.cluster import KMeans 

kmmodel = KMeans(n_clusters=2,n_init=10 ,random_state=5)

kmmodel.fit_predict(svmtrain_trans)
#print(kmmodel.labels_)

array([1, 1, 1, ..., 1, 1, 1], dtype=int32)

In [21]:
#predict on transformed test data
kmmpredict = kmmodel.predict(svmtest_trans)

print(classification_report(test['1'], kmmpredict))
print("accuracy: {0}".format(metrics.accuracy_score(test['1'], kmmpredict)))

              precision    recall  f1-score   support

           0       0.58      0.39      0.46     12500
           1       0.54      0.72      0.62     12500

    accuracy                           0.55     25000
   macro avg       0.56      0.55      0.54     25000
weighted avg       0.56      0.55      0.54     25000

accuracy: 0.55208


In [28]:
#Testing Kmeans on a Negative Review
review = """I received defective piece display is not working properly"""
review_vector = kmvectorizer.transform([review]) # vectorizing

result = int(kmmodel.predict(review_vector))

if result != 1:
  print("Negative")
else:
  print("Positive")

Positive


In [30]:
#Testing Kmeans on a Positive Review
review = """I've recently moved into a new apartment and needed a router to attach to the internet. Frankly, I was intimidated by the sheer amount (and price) of routers on the market so it felt like a gamble getting this, but I'm glad I did! It was easy to set up, the only trouble I ever had was plugging the ethernet into the wrong outlet (which was completely my bad). The signal works amazing, we have it on the main floor and get signal on both the lower and upper levels. We've had no connection issues, and everything has run perfectly so far. If you're looking for something inexpensive but functional, I'd definitely recommend this!"""
review_vector = kmvectorizer.transform([review]) # vectorizing

result = int(kmmodel.predict(review_vector))

if result != 1:
  print("Negative")
else:
  print("Positive")

Positive


In [31]:
#import Negative and Positive words files
import pandas as pd

positive = pd.read_csv('positive-words.txt', header = None)
negative = pd.read_csv('negative-words.txt', header = None)
print(positive)
print(negative)

              0
0            a+
1        abound
2       abounds
3     abundance
4      abundant
...         ...
2001   youthful
2002       zeal
2003     zenith
2004       zest
2005      zippy

[2006 rows x 1 columns]
               0
0        2-faced
1        2-faces
2       abnormal
3        abolish
4     abominable
...          ...
4778        zaps
4779      zealot
4780     zealous
4781   zealously
4782      zombie

[4783 rows x 1 columns]


In [32]:
#Tokenize and Clean the random positive review
review_words = review.split()
test_data = [re.sub(r'[^A-Za-z]+', '', each) for each in review_words]
print(test_data)

['Ive', 'recently', 'moved', 'into', 'a', 'new', 'apartment', 'and', 'needed', 'a', 'router', 'to', 'attach', 'to', 'the', 'internet', 'Frankly', 'I', 'was', 'intimidated', 'by', 'the', 'sheer', 'amount', 'and', 'price', 'of', 'routers', 'on', 'the', 'market', 'so', 'it', 'felt', 'like', 'a', 'gamble', 'getting', 'this', 'but', 'Im', 'glad', 'I', 'did', 'It', 'was', 'easy', 'to', 'set', 'up', 'the', 'only', 'trouble', 'I', 'ever', 'had', 'was', 'plugging', 'the', 'ethernet', 'into', 'the', 'wrong', 'outlet', 'which', 'was', 'completely', 'my', 'bad', 'The', 'signal', 'works', 'amazing', 'we', 'have', 'it', 'on', 'the', 'main', 'floor', 'and', 'get', 'signal', 'on', 'both', 'the', 'lower', 'and', 'upper', 'levels', 'Weve', 'had', 'no', 'connection', 'issues', 'and', 'everything', 'has', 'run', 'perfectly', 'so', 'far', 'If', 'youre', 'looking', 'for', 'something', 'inexpensive', 'but', 'functional', 'Id', 'definitely', 'recommend', 'this']


In [33]:
pos_words = [x[1] for x in positive.itertuples()]
neg_words = [x[1] for x in negative.itertuples()]

In [34]:
#Count the number of positive and negative words in the given review
pos_count = 0
neg_count = 0
for word in test_data:
  if word in pos_words:
    pos_count = pos_count + 1
  if word in neg_words:
    neg_count = neg_count + 1

print(pos_count)
print(neg_count)

8
4


In [35]:
#Rating on 5-Star Scale
rating = round(((5*pos_count)/(pos_count + neg_count)),2)
print(" The rating of the review is:", rating)

 The rating of the review is: 3.33
